# 🤔 Config

In [ ]:
#|default_exp utils.config

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# |hide
from nbdev.showdoc import *
from fastcore.test import test_eq
from lovely_jax.utils.misc import test_array_repr

In [ ]:
# |hide
# |export
from copy import copy
from types import SimpleNamespace
from typing import Optional, Union, Callable, TypeVar
from contextlib import contextmanager
from lovely_numpy import config as np_config

In [ ]:
# |exporti

class Config(SimpleNamespace):
    "Config"
    def __init__(self,
            precision     = 3,    # Digits after `.`
            threshold_max = 3,    # .abs() larger than 1e3 -> Sci mode
            threshold_min = -4,   # .abs() smaller that 1e-4 -> Sci mode
            sci_mode      = None, # Sci mode (2.3e4). None=auto
            indent        = 2,    # Indent for .deeper()
            color         = True, # ANSI colors in text
            deeper_width  = 9,    # For .deeper, width per level
            plt_seed      = 42,   # Sampling seed for `plot`
            fig_close     = True, # Close matplotlib Figure
            fig_show      = False,# Call `plt.show()` for `.plt`, `.chans` and `.rgb`

    ):
        super().__init__(**{k:v for k,v in locals().items() if k not in ["self", "__class__"]})

_defaults = Config()
_config = copy(_defaults)

### Defaults:

In [ ]:
# |echo: false
DocmentTbl(Config)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| precision | int | 3 | Digits after `.` |
| threshold_max | int | 3 | .abs() larger than 1e3 -> Sci mode |
| threshold_min | int | -4 | .abs() smaller that 1e-4 -> Sci mode |
| sci_mode | NoneType | None | Sci mode (2.3e4). None=auto |
| indent | int | 2 | Indent for .deeper() |
| color | bool | True | ANSI colors in text |
| deeper_width | int | 9 | For .deeper, width per level |
| plt_seed | int | 42 | Sampling seed for `plot` |
| fig_close | bool | True | Close matplotlib Figure |
| fig_show | bool | False | Call `plt.show()` for `.plt`, `.chans` and `.rgb` |

In [ ]:
# |exporti

# Allows passing None as an argument to reset the 
class _Default():
    def __repr__(self):
        return "Ignore"
D = _Default()
Default = TypeVar("Default")

In [ ]:
# |export

def set_config( precision       :Optional[Union[Default,int]]  =D,
                threshold_min   :Optional[Union[Default,int]]  =D,
                threshold_max   :Optional[Union[Default,int]]  =D,
                sci_mode        :Optional[Union[Default,bool]] =D,
                indent          :Optional[Union[Default,bool]] =D,
                color           :Optional[Union[Default,bool]] =D,
                deeper_width    :Optional[Union[Default,int]]  =D,
                plt_seed        :Optional[Union[Default,int]]  =D,
                fig_close       :Optional[Union[Default,bool]] =D,
                fig_show        :Optional[Union[Default,bool]] =D
                ) -> None:

    "Set config variables"
    args = locals().copy()
    for k,v in args.items():
        if v != D:
            if v is None:
                setattr(_config, k, getattr(_defaults, k))
            else:
                setattr(_config, k, v)

In [ ]:
# |export
def get_config():
    "Get a copy of config variables"
    return copy(_config)

In [ ]:
# |export
@contextmanager
def config( precision       :Optional[Union[Default,int]]   =D,
            threshold_min   :Optional[Union[Default,int]]   =D,
            threshold_max   :Optional[Union[Default,int]]   =D,
            sci_mode        :Optional[Union[Default,bool]]  =D,
            indent          :Optional[Union[Default,bool]]  =D,
            color           :Optional[Union[Default,bool]]  =D,
            deeper_width    :Optional[Union[Default,int]]   =D,
            plt_seed        :Optional[Union[Default,int]]   =D,
            fig_close       :Optional[Union[Default,bool]]  =D,
            fig_show        :Optional[Union[Default,bool]]  =D
            ):


    "Context manager for temporarily setting printting options."
    global _config
    new_opts = { k:v for k, v in locals().items() if v != D }
    old_opts = copy(get_config().__dict__)

    try:
        set_config(**new_opts)
        yield
    finally:
        set_config(**old_opts)

## Examples

In [ ]:
# |hide

# In the next cell I'm importing the functios from the exported .py as part of the examples code
# This overrides the functions defined above!
# Make sure the code is in sync

import nbdev; nbdev.nbdev_export()

In [ ]:
import jax.numpy as jnp
from lovely_jax import set_config, get_config, config, monkey_patch

In [ ]:
monkey_patch()

### Precision

In [ ]:
set_config(precision=5)
jnp.array([1., 2, float("nan")])

Array[3] μ=1.50000 σ=0.70711 NaN! gpu:0 [1.00000, 2.00000, nan]

In [ ]:
jnp.array([1., 2, float("nan")])

Array[3] μ=1.50000 σ=0.70711 NaN! gpu:0 [1.00000, 2.00000, nan]

In [ ]:
# |hide
test_array_repr(str(jnp.array([1., 2, float("nan")])),
        'Array[3] μ=1.50000 σ=0.70711 \x1b[31mNaN!\x1b[0m gpu:0 [1.00000, 2.00000, nan]')

### Scientific mode

In [ ]:
set_config(sci_mode=True) # Force always on
str(jnp.array([1., 2, float("nan")]))

'Array[3] μ=1.50000e+00 σ=7.07107e-01 \x1bNaN!\x1b gpu:0 [1.00000e+00, 2.00000e+00, nan]'

In [ ]:
#| hide
test_array_repr(str(jnp.array([1., 2, float("nan")])),
        'Array[3] μ=1.50000e+00 σ=7.07107e-01 \x1b[31mNaN!\x1b[0m gpu:0 [1.00000e+00, 2.00000e+00, nan]')

### Color on/off

In [ ]:
set_config(color=False) # Force always off
jnp.array([1., 2, float("nan")])

Array[3] μ=1.50000e+00 σ=7.07107e-01 NaN! gpu:0 [1.00000e+00, 2.00000e+00, nan]

In [ ]:
test_array_repr(str(jnp.array([1., 2, float("nan")])),
        'Array[3] μ=1.50000e+00 σ=7.07107e-01 NaN! gpu:0 [1.00000e+00, 2.00000e+00, nan]')

### Control .deeper

In [ ]:
set_config(deeper_width=3) 
image = jnp.load("mysteryman.npy")
image = image.at[1,100,100].set(float('nan'))

image.deeper(2)

Array[3, 196, 196] n=115248 x∈[-2.11790e+00, 2.64000e+00] μ=-3.88310e-01 σ=1.07319e+00 NaN! gpu:0
  Array[196, 196] n=38416 x∈[-2.11790e+00, 2.24891e+00] μ=-3.24352e-01 σ=1.03586e+00 gpu:0
    Array[196] x∈[-1.91241e+00, 2.24891e+00] μ=-6.73483e-01 σ=5.20629e-01 gpu:0
    Array[196] x∈[-1.86103e+00, 2.16328e+00] μ=-7.38488e-01 σ=4.17012e-01 gpu:0
    Array[196] x∈[-1.75828e+00, 2.19753e+00] μ=-8.05501e-01 σ=3.95835e-01 gpu:0
    ...
  Array[196, 196] n=38416 x∈[-1.96569e+00, 2.42857e+00] μ=-2.73903e-01 σ=9.72665e-01 NaN! gpu:0
    Array[196] x∈[-1.86064e+00, 2.41106e+00] μ=-5.28772e-01 σ=5.54540e-01 gpu:0
    Array[196] x∈[-1.82563e+00, 2.35854e+00] μ=-5.61732e-01 σ=4.71564e-01 gpu:0
    Array[196] x∈[-1.75560e+00, 2.37605e+00] μ=-6.21756e-01 σ=4.57265e-01 gpu:0
    ...
  Array[196, 196] n=38416 x∈[-1.80444e+00, 2.64000e+00] μ=-5.66673e-01 σ=1.17775e+00 gpu:0
    Array[196] x∈[-1.71730e+00, 2.39599e+00] μ=-9.81537e-01 σ=3.49106e-01 gpu:0
    Array[196] x∈[-1.75216e+00, 2.32627e+00] μ=-

In [ ]:
# test_eq(len(str(image.deeper(2))), 1197)

### Reser to defaults

In [ ]:
set_config(precision=None, sci_mode=None, color=None, deeper_width=None)
str(jnp.array([1., 2, float("nan")]))

'Array[3] μ=1.500 σ=0.707 \x1bNaN!\x1b gpu:0 [1.000, 2.000, nan]'

In [ ]:
test_array_repr(str(jnp.array([1., 2, float("nan")])),
    'Array[3] μ=1.500 σ=0.707 \x1b[31mNaN!\x1b[0m gpu:0 [1.000, 2.000, nan]')

### Context manager

In [ ]:
display(jnp.array([1., 2, jnp.nan]))
with config(sci_mode=True, color=False):
    display(jnp.array([1., 2, jnp.nan]))
display(jnp.array([1., 2, jnp.nan]))

Array[3] μ=1.500 σ=0.707 NaN! gpu:0 [1.000, 2.000, nan]

Array[3] μ=1.500e+00 σ=7.071e-01 NaN! gpu:0 [1.000e+00, 2.000e+00, nan]

Array[3] μ=1.500 σ=0.707 NaN! gpu:0 [1.000, 2.000, nan]

### Matplotlib and seed

In [ ]:
# torch.manual_seed(42)
# a = torch.randn(1000)

In [ ]:
# _ = a.plt() # The figure was closed, nothing is displayed

In [ ]:
# set_config(fig_close=False)
# _ = a.plt() # figure was not closed. All figures that are not closed are displayed after the cell runs.

In [ ]:
# # |hide
# set_config(fig_close=None)

For performance reasons, <code>.plt</code> will randomly sample up tp `max_s` elements from the data (10k be default).

You can change the seed used for this sampling (42 by default):

In [ ]:
# set_config(plt_seed=1)
# a.plt(max_s=100)

In [ ]:
# set_config(plt_seed=2)
# a.plt(max_s=100)

More details in [matplotlib](matplotlib.html)